# 8장 - 감성 분석에 머신 러닝 적용

# 텍스트 처리용 IMDb 영화 리뷰 데이터 준비

이따금 의견 분석(opinion minimg)이라고도 하는 감성 분석은 광범위한 NLP 연구 분야 중 인기 있는 하위 분야이다.  
감성 분석에서 인기 있는 작업은 특정 주제에 대해 작가가 표현한 의견이나 감정을 기반으로 문서를 분류하는 것이다.  

이 장에서 마스(Maas) 등이 수집한 대규모 IMBd의 영화 리뷰 데이터셋을 사용하겠다.  
이 영화 리뷰 데이터셋은 긍정 또는 부정으로 레이블되어 있는 영화 리뷰 5만개로 구성되어 있다. 여기서 긍정이란 IMDb에서 별 여섯 개 이상을 받은 영화를 말한다. 부정은 IMDb에서 별 다섯 개 아래를 받은 영화를 말한다. 

## 영화 리뷰 데이터셋 구하기

In [1]:
# 파이썬에서 다운로드하고 압축을 푸는 코드
import os
import sys
import tarfile
import time
import urllib.request


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size

    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    urllib.request.urlretrieve(source, target, reporthook)

In [2]:
# aclImdb 디렉토리에 압축을 품
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

## 영화 리뷰 데이터셋을 더 간편한 형태로 전처리

In [6]:
import pyprind
import pandas as pd
import os
import sys

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는
# 디렉토리로 바꾸세요

basepath = '/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stderr)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

/var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/ipykernel_7744/3961377723.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt, labels[l]]],
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:47


In [7]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [8]:
# 만들어진 데이터를 csv파일로 저장.
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [5]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [6]:
df.shape

(50000, 2)

# BoW 모델 소개

텍스트나 단어 같은 범주형 데이터를 머신 러닝 알고리즘에 주입하기 전에 수치 형태로 변환해야 한다고 언급했다.  
이 절에서 텍스트를 수치 특성 벡터로 표현하는 **BoW**(Bag-of-Word)를 소개한다. BoW 모델의 아이디어는 매우 간단하며 다음과 같이 정리할 수 있다.  
1. 전체 문서에 대해 고유한 토큰(token), 에를 들어 단어로 이루어진 어휘 사전(vocabulary)을 만든다. 
2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만든다.  

각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분에 지나지 않으므로 특성 벡터는 대부분이 0으로 채워진다. 그래서 이 트겅 벡터는 **희소**(sparse)하다고 한다. 

## 단어를 특성 벡터로 변환

`CountVertorizer`의 fit_transform 메서드를 호출하여 BoW 모델의 어휘사전을 만들고 다음 세 문장을 희소한 특성 벡터로 변환.  
- The sun is shining
- The weather is sweet
- The sun is shining, the weather is sweet, and one and one is two

In [7]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [8]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [9]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


특성 벡터의 각 인덱스는 `CountVectorizer`의 어휘 사전 딕셔너리에 저장된 정수 값에 해당된다.  
예를 들어 인덱스 0에 있는 첫 번째 특성은 'and' 단어의 카운트를 의미한다. 이 단어는 마지막 문서에만 나타난다.  
인덱스 1에 있는 (특성 벡터의 두 번째 열)단더 'is'는 세 문장에 모두 등장한다. 특성 벡터의 이런 값들을 단어 빈도(terms frequency)라고도 한다. 문서 d에 등장한 단어 t의 횟수를 $tf(t,d)$와 같이 쓴다.  
BoW 모델은 문장이나 문서에 등장하는 단어의 순서를 상관하지 않는다. 특성 벡터에 나타나는 단어 빈도의 순서는 보통 어휘 사전의 알파벳 순서를 따른다. 

## tf-idf를 사용하여 단어 적합성 평가

텍스트 데이터를 분석할 때 클래스 레이블이 다른 문서에 같은 단어들이 나타나는 경우를 종종 보게 된다.  
일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않다.  
이 절에서는 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법인 **tf-idf**(term frequency-inverse document frequency)를 배우겠다.  
tf-idf는 단어 빈도와 역문서 빈도(inverse document frequency)의 곱으로 정의된다.  
$$tf-idf(t,d)=tf(t,d) \times ids(t,d)$$  

여기서 $tf(t,d)$는 이전 절에서 보았던 단어 빈도이다. $idf(t,d)$는 역문서 빈도로 다음과 같이 계산된다.  
$$idf(t,d)=log({n_{d} \over 1+df(d,t)})$$  

여기서 $n_{d}$는 전체 문서 개수이고 $df(d,t)$는 단어 t가 포함된 문서 d의 개수이다. 분모에 상수 1을 추가하는 것은 선택 사항이다. log는 무서 빈도 $df(d,t)$가 낮을 때 역문서 빈도 값이 너무 커지지 않도록 만든다.

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


이전 절에서 보았듯이 세 번째 문서에서 단어 'is'가 가장 많이 나타났기 때문에 단어 빈도가 가장 컸다.  
동일한 특성 벡터를 tf-idf로 변환하하면 단어 'is'는 비교적 작은 tf-idf를 가진다. 이 단어는 첫 번째와 두 번째 문서에도 나타나므로 판별에 유용한 정보를 가지고 있지 않을 것이다.  

수동으로 특성 벡터에 있는 각 단어의 tf-idf를 계산해보면 `TfidfTransformer`가 앞서 정의한 표준 tf-idf 공식과 조금 다르게 계산한다는 것을 알 수 있다. 사이킷런에 구현된 역문서 빈도 공식은 다음과 같다.  
$$idf(t,d)=log{1+n_{d} \over {1+df(t,d)}}$$  

비슷하게 사이킷런에서 계산하는 tf-idf는 앞서 정의한 공식과 조금 다르다.  
$$tf-idf(t,d)=tf(t,d) \times (idf(t,d)+1)$$  
앞의 식에서 '+1'을 더한 것은 모든 문서에 등장하는 단어가 가중치 0이 되는 것(즉, idf(t,d)=log(1)=0)을 막기 위해서이다.  

일반적으로 tf-idf를 계산하기 전에 단어 빈도(tf)를 정규화하지만 `TfidfTransformer` 클래스는 tf-idf를 직접 정규화한다. 사이킷런의 `TfidfTransformer`는 기본적으로 L2 정규화를 적용한다. 

## 텍스트 데이터 정제

BoW 모델을 만들기 전에 첫 번째로 수행할 중요한 단계는 불필요한 문자를 삭제하여 텍스트 데이터를 정제하는 일이다. 

In [11]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

여기서 볼 수 있듯이 HTML 마크업(markup)은 물론 구두점과 글자가 아닌 문자가 포함되어 있다. HTML 마크업에는 유용한 의미가 많지 않지만 구두점은 특정 NLP 문제에서 쓸 모 있는 추가 정보가 될 수 있다.  
여기서는 문제를 간단하게 만들기 위해 :)같은 문자를 제외하고 모든 구두점 기호를 삭제한다. 이런 이모티콘은 확실히 감성 분석에 유용하기 때문이다.  
파이썬의 **정규 표현식**(regular expression) 라이브러리 re를 사용하여 이런 작업을 수행한다.

In [12]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text) # '<[^>]*>'는 '<' 괄호로 시작해서 '>'괄호가 나올 때까지 '>'괄호가 아닌 문자를 0개 이상(*) 매칭한다는 뜻이다. 
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) + # '[\W]'ㄴ,ㄴ '[^A-Za-z0-9]+'와 동일하다.
            ' '.join(emoticons).replace('-', ''))
    return text

In [13]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [14]:
preprocessor("This :) is :( a test :-)!")

'this is a test :) :( :)'

In [15]:
df['review'] = df['review'].apply(preprocessor)

## 문서를 토큰으로 나누기

영화 리뷰 데이터셋을 전처리한 후에는 어떻게 텍스트 문서를 낱개의 토큰으로 나눌지 생각해야 한다. 문서를 토큰화하는 한 가지 방법은 공백 문자를 기준으로 개별 단어로 나누는 것이다.

In [17]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

토큰화 방법 중에는 단어를 변하지 않는 기본 형태인 어간으로 바꾸는 **어간 추출**(stemming)이란 다른 방법이 있다. 여러 가지 형태를 갖는 단어를 같은 어간으로 매핑할 수 있다.  
초기 어간 추출 알고리즘은 1979년 마틴 포터에 의해 개발되었고 **포터 어간 추출기**(Porter stemmer) 알고리즘이라고 불린다. 파이썬 **NLTK** 패키지에 포터 어간 추출 알고리즘이 구현되어 있다. 

In [18]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


In [19]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [20]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

nltk 패키지의 PorterStemmer 클래스를 사용하여 단어의 어간으로 바꾸기 위해 tokenizer 함수를 변경했다. 앞 예를 보면 단어 'running'이 어간 'run'으로 바뀌었다.   

BoW 모델을 사용하여 머신 러닝 모델을 훈련시키기 전에 **불용어**(stop-word)제거에 대해 짧게 소개한다.  
불용어는 모든 종류의 텍스트에 아주 흔하게 등장하는 단어이다. 불요어에는 문서의 종류를 구별하는 데사용할 수 있는 정보가 없거나 아주 조금만 있다. 불용어 예로는 is, and, has, like 등이 있다.  
불용어 제거는 tf-idf보다 기본 단어 빈도나 정규화된 단어 빈도를 사용할 때 더 유용하다. tf-idf에는 자주 등장하는 단어의 가중치가 이미 낮추어져 있다. 

In [21]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhyeongu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
if w not in stop]


['runner', 'like', 'run', 'run', 'lot']

# 문서 분류를 위한 로지스틱 회귀 모델 훈련

이 절에서 BoW 모델을 기반으로 영화 리뷰를 긍정과 부정 리뷰로 분류하는 로지스틱 회귀 모델을 훈련시켜보자. 

In [23]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

HalvingGridSearchCV 객체에서 5-겹 계층별 교차 검증을 사용하여 로지스틱 회귀 모델에 대한 최적의 매개변수 조합을 찾는다. 

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='liblinear'))])

hgs_lr_tfidf = HalvingGridSearchCV(
                           estimator = lr_tfidf, 
                           param_grid = param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1)

In [29]:
import warnings
warnings.filterwarnings('ignore')
hgs_lr_tfidf.fit(X_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. T

HalvingGridSearchCV(estimator=Pipeline(steps=[('vect',
                                               TfidfVectorizer(lowercase=False)),
                                              ('clf',
                                               LogisticRegression(random_state=0,
                                                                  solver='liblinear'))]),
                    n_jobs=-1,
                    param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                                 'clf__penalty': ['l1', 'l2'],
                                 'vect__ngram_range': [(1, 1)],
                                 'vect__stop_words': [['i', 'me', 'my',
                                                       'myself', 'we', 'our',
                                                       'ours', 'ourselves',
                                                       'you', "you're",
                                                       "you'...
                                                      

In [30]:
print('최적의 매개변수 조합: %s ' % hgs_lr_tfidf.best_params_)
print('CV 정확도: %.3f' % hgs_lr_tfidf.best_score_)

최적의 매개변수 조합: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7faab9405f70>} 
CV 정확도: 0.897


In [31]:
clf = hgs_lr_tfidf.best_estimator_
print('테스트 정확도: %.3f' % clf.score(X_test, y_test))

테스트 정확도: 0.899


# 대용량 데이터 처리: 온라인 알고리즘과 외부 메모리 학습

이전 절의 예제를 실행하려면 그리드 서치 안에서 5만 개의 영화 리뷰를 위한 특성 벡터를 만드는데 계산 비용이 많이 소요된다는 것을 알 수 있다. 많은 실전 에플리케이션에서는 심지어 컴퓨터 메모리를 초과하는 대량의 데이터를 다루는 경우가 드물지 않다.  
대량의 데이터셋을 다룰 수 있는 외부 메모리 학습(out-of-core learning) 기법을 사용한다.  
이 방법은 데이터셋을 작은 배치(batch)로 나누어 분류기를 점진적으로 학습시킨다.  

한 번에 샘플 하나를 사용하여 모델의 가중치를 업데이트하는 최적화 알고리즘인 **확률적 경사 하강법**(stochastic gradient descent)을 소개했다. 이 절에서는 사이킷런에 있는 SGDClassifier의 partial_fit 메서드를 사용하여 로지스틱 휘귀 모델을 훈련하겠다.  
이를 위해 로컬 디스크에서 문서를 직접 읽어 작은 크기의 미니 배치(mini-batch)로 만든다.

In [40]:
import numpy as np
import re
from nltk.corpus import stopwords


# `stop` 객체를 앞에서 정의했지만 이전 코드를 실행하지 않고
# 편의상 여기에서부터 코드를 실행하기 위해 다시 만듭니다.
stop = stopwords.words('english')


def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

# 한 번에 문서 하나씩 읽어서 반환하는 stream_docs 제너레이터 함수를 정의
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # 헤더 넘기기
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [41]:
next(stream_docs(path='/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

이제 stream_docs 함수에서 문서를 읽어 size 매개변수에서 지정한 만큼 문서를 변환하는 get_minibatch 함수를 정의

In [42]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

외부 메모리 학습에 `CountVectorizer` 클래스를 사용할 수 없다. 이 클래스는 전체 어휘 사전을 메모리에 가지고 있어야 하기 때문이다. 또 `TfidfVectorizier` 클래스는 역문서 빈도를 계산히기 위해 훈련 데이터셋의 특성 벡터를 모두 메모리에 가지고 있어야 한다.  
사이킷런에서 텍스트 처리에 사용할 수 있는 다른 유용한 클래스는 `HashingVectorizer`이다. `HashingVectorizier`는 데이터 종류에 상관없이 사용할 수 있다.

In [43]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier


vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

In [44]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

clf = SGDClassifier(loss='log', random_state=1)


doc_stream = stream_docs(path='/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/movie_data.csv')

앞 코드에서 tokenizer 함수와 특성 개수를 2**21로 설정하여 HashingVectorizer 클래스를 초기화했다. 또 SGDClassifier 클래스의 loss 매개변수를 'log'로 지정하여 로지스틱 휘귀 모델로 초기화한다. HashinVectorizer에서 특성 개수를 크게 하면 해서 충돌 가능성을 줄일 수 있지만 로지스틱 회귀 모델의 가중치 개수도 늘어난다. 

In [45]:
import pyprind
pbar = pyprind.ProgBar(45,stream=sys.stderr)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


진행 막대 객체를 45번 반복으로 초기화하고 이어지는 for 반복문에서 45개의 미니 배치를 반복한다. 각 미니 배치는 1000개의 문서로 구성된다. 점진적인 학습 과정이 끝나면 마지막 5000개의 문서를 사용하여 모델의 성능을 평가한다.

In [46]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [47]:
clf = clf.partial_fit(X_test, y_test)

결과에서 볼 수 있듯이 모델의 정확도는 약 87%이다.  
이전 절에서 그리드 서치로 하이퍼파라미터 튜닝을 하여 달성한 정확도보다 조금 낮다. 그렇지만 외부 메모리 학습은 매우 메모리 효율적이고 모델 훈련이 채 1분도 걸리지 않는다. 마지막 나머지 5000개의 문서를 사용하여 모델을 업데이트한다. 

# 잠재 디리클레 할당을 사용한 토픽 모델링

토픽 모델링(topic modeling)은 레이블이 없는 텍스트 문서에 토픽을 할당하는 광범위한 분야이다. 예로 들 수 있는 전형적인 애플리케이션은 대량의 뉴스 기사 데이터셋을 분류하는 일이다. 토픽 모델링 애플리케이션은 이런 기사에 카테고리 레이블을 할당한다.  

이 절에서 인기 있는 토픽 모델링 기법인 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)을 소개한다. 5장에서 소개한 지도 방식의 차원 축소 기법인 선형 판별 분석(LDA)과 혼동하지 않길 바란다. 

## LDA를 사용한 텍스트 문서 분해

LDA 이면의 수학은 꽤 복잡하고 베이지안 추론(Bayesian inference)에 관한 지식이 필요하다. 여기서는 이 주제를 엔지니어 관점에서 접근하여 일반적인 용어를 사용해서 LDA를 이해해본다.  

LDA는 여러 문서에 걸쳐 자주 등장하는 단어의 그룹을 찾는 확률적 생성 모델이다. 각 문서를 여러 단어가 혼합된 것으로 가정하면 토픽은 자주 등장하는 단어들로 나타낼 수 있다. LDA의 입력은 앞서 보았던 BoW 모델이다. LDA는 입력으로 받은 BoW 행렬을 두 개의 행렬로 분해한다. 
- 문서-토픽 행렬
- 단어-토픽 행렬  

이 두 행렬을 곱해서 가능한 작은 오차로 BoW 입력 행렬을 재구성할 수 있도록 LDA가 BoW 행렬을 분해한다. 실제로는 LDA가 BoW 행렬에서 찾은 토픽이 관심 대상이다. 유일한 단점은 미리 토픽 개수를 정해야 한다는 것이다. 즉, 토픽 개수는 LDA의 하이퍼파라미터로 수동으로 지정해야 한다. 

## 사이킷런의 LDA

In [48]:
import pandas as pd

df = pd.read_csv('/Users/hanhyeongu/Desktop/HG/code study/ML_DL_study/ch08/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [49]:
# 그다음 이제 익숙한 countVectorizier 클래스를 사용하여 BoW 행렬 생성
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

단어의 최대 문서 빈도를 10%로 지정하여(max_df=.1) 여러 문서에 걸쳐 너무 자주 등장하는 단어를 제외함.  
자주 등장하는 단어를 제외하는 이유는 모든 문서에 걸쳐 등장하는 단어일 수 있고, 그런 단어는 문서의 특정 토픽 카테고리와 관련성이 적기 때문이다.  
또 가장 자주 등장하는 단어 5000개로 단어 수를 제한했다.(max_features=5000) 이는 데이터셋의 차원을 제한하여 LDA의 추론 성능을 향상시킨다. 

In [50]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)

사이킷런에 구현된 LatentDirichletAllocation 클래스를 사용하여 영화 리뷰 데이터셋을 분해하고 여러 개의 토픽으로 분류. 열 개의 토픽으로 한정하여 분석.   

learning_method='batch'로 설정햇으므로 lda 추정기가 한 번 반복할 때 가능한 모든 훈련 데이터(BoW 행렬)를 사용하여 학습된다. 'online' 설정보다 느리지만 더 정확한 결과를 만든다. 

In [51]:
'''
LDA를 학습하고 나면 lda 객체의 components_ 속성에 열 개의 토픽에 대해 
오름차순으로 단어의 중요도를 담은 행렬이 저장된다. 
'''
lda.components_.shape

(10, 5000)

In [52]:
n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


결과를 분석하기 위해 열 개의 토픽에서 가장 중요한 단어를 다섯 개씩 출력해보았다.  

각 토픽에서 가장 중요한 단어 다섯 개를 기반으로 LDA가 다음 토픽을 구별했다고 추측할 수 있다.  
1. 대체적으로 형편없는 영화(실제 토픽 카데고리가 되지 못함)
2. 가족 영화
3. 전쟁 영화
4. 예술 영화
5. 범죄 영화
6. 공포 영화
7. 코미디 영화
8. TV 쇼와 관련된 영화
9. 소설을 원작으로 한 영화
10. 액션 영화  

카테고리가 잘 선택됐는지 확인하기 위해 공포 영화 카테고리에서 3개 영화의 리뷰를 출력해보자.

In [53]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\n공포 영화 #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


공포 영화 #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

공포 영화 #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

공포 영화 #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...


앞 코드에서 공포 영화 카테고리 중 최상위 3개의 리뷰에서 300자씩 출력. 정확히 어떤 영화에 속한 리뷰인지는 모르지만 공포 영화의 리뷰임을 알 수 있다. (하지만 영화 #2는 1번 카테고리에 속하는 것처럼 보이기도 한다.)